# Spectral Analysis for Modal Parameter Linear Estimate

## Setup
### Libraries

In [ ]:
# Install requirements
import sys
!$sys.executable -m pip install -r ../requirements.txt --upgrade

from IPython import display
display.clear_output()

In [ ]:
# Add parent directory to path
import os
sys.path.insert(0, os.path.abspath(".."))

In [ ]:
import sample
from tests import utils
import numpy as np
from matplotlib import pyplot as plt
import librosa.display

### Generate test audio

In [ ]:
fs = 44100
x = utils.test_audio(fs=fs)

librosa.display.waveplot(x, sr=fs, zorder=100)
plt.grid()
plt.gcf().set_size_inches([12, 6])
display.Audio(x, rate=fs)

## Sinusoidal Model

In [ ]:
from sample import SAMPLE
sample = SAMPLE(
    sinusoidal_model__max_n_sines=10,
    sinusoidal_model__peak_threshold=-30,
    sinusoidal_model__save_intermediate=True
).fit(x)

### STFT

In [ ]:
stft = np.array([mx for mx, _ in sample.sinusoidal_model.intermediate_["stft"]]).T

librosa.display.specshow(stft, sr=fs, x_axis="time", y_axis="hz");
plt.ylim([0, 2000])
plt.gcf().set_size_inches([12, 6])

### Peak detection

In [ ]:
mx, px = sample.sinusoidal_model.intermediate_["stft"][0]
f = fs * np.arange(mx.size) / sample.sinusoidal_model.w_.size
ploc, pmag, pph = sample.sinusoidal_model.intermediate_["peaks"][0]

ax = plt.subplot(121)
plt.fill_between(f, np.full(mx.shape, -120), mx, alpha=.1)
plt.plot(f, mx)
plt.scatter(ploc * fs / sample.sinusoidal_model.w_.size, pmag, c="C0")
plt.ylim([-60, plt.ylim()[1]])
plt.grid()
plt.title("magnitude")

plt.subplot(122, sharex=ax)
plt.plot(f, px)
plt.scatter(ploc * fs / sample.sinusoidal_model.w_.size, pph)
plt.ylim([np.min(px[f < 2000]), np.max(px[f < 2000])])
plt.grid()
plt.title("phase")
plt.xlim([0, 2000])
plt.gcf().set_size_inches([12, 6])

### Sine tracking

In [ ]:
from sample import plots
plots.sine_tracking_2d(sample.sinusoidal_model)
plt.gcf().set_size_inches([12, 6])

In [ ]:
from sample import plots
plots.sine_tracking_3d(sample.sinusoidal_model)
plt.gcf().set_size_inches([12, 6])

## Regression

In [ ]:
_, ax = plt.subplots(1, 2, sharex=True, sharey=True)
plots.sine_tracking_2d(sample.sinusoidal_model, ax=ax)

t_x = np.arange(x.size) / fs
ax[0].cla()
for f, d, a in sample.param_matrix_.T:
    ax[0].plot(t_x, 20*np.log10(a * np.exp(-2*t_x / d)))

ax[0].set_title("fitted curves")
ax[0].grid()
ax[0].set_ylabel("magnitude (dB)")
ax[0].set_xlabel(ax[1].get_xlabel())
ax[1].set_title("ground truth")
ax[1].set_ylabel("")

plt.gcf().set_size_inches([12, 6])

# Resynthesize

In [ ]:
x_hat = np.squeeze(sample.predict(np.arange(x.size) / fs))

librosa.display.waveplot(x_hat, sr=fs, zorder=100)
plt.grid()
plt.gcf().set_size_inches([12, 6])
display.Audio(x_hat, rate=fs)

Play against each other in stereo

In [ ]:
x_dual = np.array([x, x_hat])
for l, xi in zip(("original", "resynthesis"), x_dual):
    librosa.display.waveplot(xi, sr=fs, zorder=100, label=l)
plt.grid()
plt.legend()
plt.gcf().set_size_inches([12, 6])
display.Audio(x_dual, rate=fs)